# Sheet 2

In [23]:
# Imports
import json
import pandas as pd
import numpy as np

# Excersise 1a
You will notice that this format is not particularly suited for data analysis. Bring the data into a better format, transform and reshape the data.

In [42]:
with open('./data/spotify_modified.json') as f:
    data = json.load(f)

df = pd.json_normalize(data)

df


,track_name,artist(s)_name,artist_count,release_date,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,in_apple_charts,in_deezer_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,07/14/23,553,147,141381703,43,263,45,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,03/23/23,1474,48,133716286,48,126,58,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,06/30/23,1397,113,140003974,94,207,91,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,08/23/19,7858,100,800840817,116,207,125,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,05/18/23,3133,50,303236322,84,133,87,...,144,A,Major,65,23,80,14,63,11,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,Sweet Nothing,Taylor Swift,1,10/21/22,1747,0,186104310,9,6,13,...,177,,Major,34,39,16,97,0,12,5
954,Cruel Summer,Taylor Swift,1,08/23/19,7858,100,800840817,116,207,125,...,170,A,Major,55,58,72,11,0,11,15
955,QUEMA,"Sog, Ryan Castro, Peso Pluma",3,07/13/23,437,31,11599388,17,29,26,...,97,,Major,79,92,89,5,0,6,5
956,Titi Me Pregunt,Bad Bunny,1,05/06/22,9037,42,1264310836,124,133,139,...,107,F,Minor,65,19,72,10,0,13,25


# Excersise 1b
Are there outliers in the danceability (danceability_%)?
- Please use several outlier detection methods to justify your answer. 
- How do the outlier-detection algorithms and their results differ?

In [25]:
df['danceability_%'].describe()


count    958.000000
mean      66.862213
std       16.034765
min      -60.000000
25%       57.000000
50%       69.000000
75%       78.000000
max      199.000000
Name: danceability_%, dtype: float64

In [26]:
sorted(df['danceability_%'].unique())

[-60,
 -12,
 23,
 24,
 25,
 27,
 28,
 29,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 110,
 199]

In [27]:
df['danceability_%'].isna().sum()

0

## Z-Score
Identifies outliers more than 3 standard deviations from the mean.


In [28]:
df['z_score'] = (df['danceability_%'] - df['danceability_%'].mean()) / df['danceability_%'].std()

outliers = df[df['z_score'].abs() > 3]

print(outliers['danceability_%'])

92     -12
224    199
666    -60
Name: danceability_%, dtype: int64


## Interquartile Range
Finds outliers outside the range between the 25th and 75th percentiles

In [29]:
Q1 = df['danceability_%'].quantile(0.25)
Q3 = df['danceability_%'].quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['danceability_%'] < lower_bound) | (df['danceability_%'] > upper_bound)]

print(outliers['danceability_%'])


46     110
92     -12
224    199
387     25
447     24
469     23
666    -60
Name: danceability_%, dtype: int64


## Absolute Deviation about Median
Identifies outliers by finding values that deviate more than 3 times the median absolute deviation from the median
- Z-Score using mean and MAD is using median

In [30]:
median = df['danceability_%'].median()

mad = 1.4826 * np.median(np.abs(df['danceability_%'] - median))

lower_bound = median - 3 * mad
upper_bound = median + 3 * mad

outliers = df[(df['danceability_%'] < lower_bound) | (df['danceability_%'] > upper_bound)]

print(outliers['danceability_%'])


92     -12
224    199
447     24
469     23
666    -60
Name: danceability_%, dtype: int64


## Interpretation of Results
### Differ of Algorithms
- Z-Score is based on the mean and standard deviation, so it is sensitive to outliers and works best for normally distributed data.
- Interquartile Range looks at the range between the 25th and 75th percentiles.
- Absolute Deviation about Median uses the median and the absolute differences to identify outliers

### Differentiation of results
- There are 4 valid outliers: (-60;-12;110;199)
- Every method found at least three of them
- The Interquartile Range found all, but got also three values which seems to be valid: 23;24;25

# Excersise 1c
There are some other anomalies in the dataset. Find them, document your findings, handle them appropriately and describe which method you applied.

In [31]:
cleaned_df = df.copy()

In [32]:
df.describe().round(2)

,in_apple_playlists,in_apple_charts,in_deezer_playlists,in_deezer_charts,bpm,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,z_score
count,958.00,958.00,958.00,958.00,958.00,958.00,958.00,958.00,958.00,958.00,958.00,958.00,958.00
mean,67.82,52.12,383.54,2.68,125.92,66.86,51.42,64.26,27.05,1.57,18.20,10.16,-0.00
std,86.29,50.85,1127.82,6.04,58.68,16.03,23.49,16.61,26.06,8.39,13.70,9.91,1.00
min,0.00,0.00,0.00,0.00,65.00,-60.00,4.00,9.00,0.00,0.00,3.00,2.00,-7.91
25%,13.00,7.00,13.00,0.00,100.00,57.00,32.25,53.00,6.00,0.00,10.00,4.00,-0.62
50%,34.00,38.50,44.00,0.00,121.00,69.00,51.00,66.00,18.00,0.00,12.00,6.00,0.13
75%,88.00,87.00,163.75,2.00,141.00,78.00,70.00,77.00,43.00,0.00,23.75,11.00,0.69
max,672.00,275.00,12367.00,58.00,999.00,199.00,97.00,97.00,97.00,91.00,97.00,64.00,8.24


In [33]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_name            958 non-null    object 
 1   artist(s)_name        958 non-null    object 
 2   artist_count          958 non-null    object 
 3   release_date          958 non-null    object 
 4   in_spotify_playlists  958 non-null    object 
 5   in_spotify_charts     958 non-null    object 
 6   streams               958 non-null    object 
 7   in_apple_playlists    958 non-null    int64  
 8   in_apple_charts       958 non-null    int64  
 9   in_deezer_playlists   958 non-null    int64  
 10  in_deezer_charts      958 non-null    int64  
 11  in_shazam_charts      958 non-null    object 
 12  bpm                   958 non-null    int64  
 13  key                   958 non-null    object 
 14  mode                  958 non-null    object 
 15  danceability_%        9

### Dancability

In [34]:
# Fix Dancabiliyt
cleaned_df = df[(df['danceability_%'] >= 0) & (df['danceability_%'] <= 100)]

### Key

In [35]:
key_counts = df['key'].value_counts()
print(key_counts)

key
C#    120
       97
G      96
G#     91
F      90
B      82
D      81
A      76
F#     73
E      62
A#     57
D#     33
Name: count, dtype: int64


In [ ]:
# Fix key column
cleaned_df['key'] = cleaned_df['key'].str.replace('#', '', regex=False).str.strip()

cleaned_df = cleaned_df[cleaned_df['key'] != '']

### BPM

In [37]:
# Due to the describe function the max of the BPM value is 999, which is not possible
top_10_bpm = df['bpm'].nlargest(10)
print(top_10_bpm)


112    999
326    999
432    888
936    777
100    206
506    206
28     204
244    202
447    202
726    200
Name: bpm, dtype: int64


In [38]:
# Fix field BPM
cleaned_df = cleaned_df[cleaned_df['bpm'] <= 250]


## Anomalies Findings
- Data Field "Key" has misspellings with "#" and one empty 97 empty values
    - Fix: Remove the "#" from the values and delete the 97 empty values

- Data Field "Dancability" also have anomalies with valous outside of [0,100] because its a percentage value
    - Fix: The 4 Values got deleted

- Data Field "BPM" has 4 Values which are too high, based on domain knowledge
    - Fix: The mean of all BPM values is written to the values

# Excersise 1d
Handle missing data appropriately. Which method did you choose and why?
## Summary of Methods:

- **Field "Key"**:
  - **Method**: manual replacement `replace('#', '')` and removal of empty values.
  - **Reason**: To correct misspellings by removing `#` and deleting 97 empty values to ensure data consistency.

- **Field "Danceability"**:
  - **Method**: Filtered values outside the [0, 100] range.
  - **Reason**: Since it’s a percentage, values outside this range were considered anomalies and removed.

- **Field "BPM"**:
  - **Method**: Replaced the excessively high BPM values with the mean.
  - **Reason**: Domain knowledge indicated the values were too high, so they were replaced with the average.



# Excersise 1e
Is there a class imbalance - if so, where and what might be the consequences?

In [39]:
key_distribution = cleaned_df['key'].value_counts(normalize=True) * 100
print("Distribution of field key: ", key_distribution, "\n")

mode_distribution = cleaned_df['mode'].value_counts(normalize=True) * 100
print("Distribution of field mode: ", mode_distribution)

Distribution of field key:  key
G    21.779859
F    18.969555
A    15.573770
C    13.934426
D    13.114754
B     9.484778
E     7.142857
Name: proportion, dtype: float64 

Distribution of field mode:  mode
Major    64.519906
Minor    35.480094
Name: proportion, dtype: float64


## Conclusion
In this dataset there is no class imbalance, the Fields "Key" and "Mode" had the opportunity to have class imbalance, but based on the distribution, there is none 